# Biotest - List
## Beachten
* Für die OCR-Erkennung wurde ABBYY Fine Reader for Mac verwendet (Lizenz vorhanden)
* Wichtig: Sprachen einstellen auf: Deutsch, Englisch, Italienisch, Französisch
* Als CSV exportieren
* HCP & HCO manuell setzen

In [52]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib
import re

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *
import consts

## Import CSV

In [53]:
# Read CSV into DataFrame
header = ['name', 'country', 'plz', 'location', 'address', 'donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'total']  
df = pd.read_csv("efpia_2017-biotest-schweiz-ag_ch.csv", sep=";", header=None, names=header)

## Format Table

In [54]:
df_export = df.copy()

#Remove rows which have no values
df_export = df_export.dropna(subset=['total'], how='all')
df_export = df_export.dropna(subset=['name'], how='all')

#Remove "Gesamtbetrag"
df_export = df_export[~df_export['name'].str.contains('Gesamtbetrag', na=False)]

#Set Type manually
index_hco = df_export[df_export['name'].str.contains("ADVISIS AG", na=False)].index[0]
df_export['type'] = np.where(df_export.index < index_hco, 'hcp', 'hco')

#Add "related expenses"
df_export.insert(3, 'related_expenses', '')

#Remove "N.A"
df_export['address'] = df_export['address'].str.replace("N.A.", '')

#Clean Numbers
df_export = cleanup_number(df_export)
for column in number_fields:
    df_export[column] = df_export[column].str.replace('O', '0')
    df_export[column] = df_export[column].str.replace('l', '1')

df_export = remove_in_numbers(df_export, '’')
df_export = amounts_to_number(df_export)

#Add UCI
df_export = add_uci(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

#Export
add_warning(manually=True)
export_list(df_export, 'biotest')

saved


In [ ]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx')